# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_enunciados.csv', sep = '|')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_ENUNCIADO,TEXTO_ENUNCIADO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995437,Não comprovada a participação do beneficiário ...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025587,A operação ficta de exportação de plataformas ...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455370,"No âmbito do TCU, a dosimetria da pena tem com...",Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773746,O princípio da independência das instâncias pe...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773402,É vedado aos dirigentes das entidades do Siste...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13312, 12)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          557
Contrato Administrativo     942
Convênio                    685
Desestatização              140
Direito Processual         1813
Finanças Públicas           328
Gestão Administrativa       339
Licitação                  2765
Pessoal                    3396
Responsabilidade           2347
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13312, 10)

In [7]:
from keras.preprocessing.text import Tokenizer
import numpy as np

limite_texto = 200
dim_vetor = 100

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_ENUNCIADO'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

sequences = tokenizer.texts_to_sequences(df['TEXTO_ENUNCIADO'])

Using TensorFlow backend.


Found 15387 unique tokens.


In [8]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13312, 200)


In [12]:
from gensim.models import Word2Vec

model = Word2Vec.load('../vocabularios/modelo-acordaos.w2v')

In [15]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, 100))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 15387
Encontrados no modelo: 7889 = 51.270553064275035


In [16]:
nok = 0
for word, i in tokenizer.word_index.items():
    if word not in model.wv:
        print(word)
        nok += 1
        if nok > 20: break

tcu
666
443
ec
bdi
vpni
ti
mp
sus
inss
stf
911
iso
cobit
lc
sicro
711
lo
1952
nbr
iec


# Treinamento

## Embedding com pesos fixos

In [17]:
from keras.layers import Embedding

embedding = Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False)

W0904 11:41:11.086553 140465010759488 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [18]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(embedding)
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

W0904 11:41:19.223565 140465010759488 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0904 11:41:19.308441 140465010759488 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0904 11:41:19.346229 140465010759488 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0904 11:41:19.347670 140465010759488 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              40962048  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10250     
Total para

In [19]:
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0904 11:41:45.129506 140465010759488 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 20s 2ms/step - loss: 1.5372 - categorical_accuracy: 0.5714 - val_loss: 1.2034 - val_categorical_accuracy: 0.5937
Epoch 2/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.8696 - categorical_accuracy: 0.7483 - val_loss: 1.0174 - val_categorical_accuracy: 0.6940
Epoch 3/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.5528 - categorical_accuracy: 0.8382 - val_loss: 1.2643 - val_categorical_accuracy: 0.6692
Epoch 4/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.3574 - categorical_accuracy: 0.8944 - val_loss: 1.4318 - val_categorical_accuracy: 0.6812
Epoch 5/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.2524 - categorical_accuracy: 0.9272 - val_loss: 1.5010 - val_categorical_accuracy: 0.6801
Epoch 6/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.1740 - categori

In [20]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 18s 2ms/step - loss: 1.2368 - categorical_accuracy: 0.6253 - val_loss: 1.2598 - val_categorical_accuracy: 0.6102
Epoch 2/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.4607 - categorical_accuracy: 0.8558 - val_loss: 1.0949 - val_categorical_accuracy: 0.6962
Epoch 3/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.1594 - categorical_accuracy: 0.9489 - val_loss: 1.5818 - val_categorical_accuracy: 0.6609
Epoch 4/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.0699 - categorical_accuracy: 0.9803 - val_loss: 2.1032 - val_categorical_accuracy: 0.6557
Epoch 5/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.0495 - categorical_accuracy: 0.9869 - val_loss: 2.2420 - val_categorical_accuracy: 0.6530
Epoch 6/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.0303 - categori

In [21]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 17s 2ms/step - loss: 1.2599 - categorical_accuracy: 0.6106 - val_loss: 1.1121 - val_categorical_accuracy: 0.6350
Epoch 2/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.6460 - categorical_accuracy: 0.7932 - val_loss: 1.1354 - val_categorical_accuracy: 0.6665
Epoch 3/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.3330 - categorical_accuracy: 0.8935 - val_loss: 1.3580 - val_categorical_accuracy: 0.6684
Epoch 4/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.1751 - categorical_accuracy: 0.9443 - val_loss: 1.5582 - val_categorical_accuracy: 0.6816
Epoch 5/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.1011 - categorical_accuracy: 0.9716 - val_loss: 2.0785 - val_categorical_accuracy: 0.6654
Epoch 6/20
10649/10649 [==============================] - 16s 2ms/step - loss: 0.0678 - categori

## Embedding com pesos ajustáveis

In [22]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0904 11:58:45.674276 140465010759488 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0904 11:58:45.699276 140465010759488 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
flatten_4 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 2048)              40962048  
_________________________________________________________________
dropout_7 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                10250     
Total para

In [23]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 19s 2ms/step - loss: 1.2292 - categorical_accuracy: 0.6262 - val_loss: 1.1266 - val_categorical_accuracy: 0.6568
Epoch 2/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.4563 - categorical_accuracy: 0.8561 - val_loss: 1.2245 - val_categorical_accuracy: 0.6541
Epoch 3/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.1393 - categorical_accuracy: 0.9541 - val_loss: 1.6815 - val_categorical_accuracy: 0.6684
Epoch 4/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.0772 - categorical_accuracy: 0.9791 - val_loss: 1.9124 - val_categorical_accuracy: 0.6748
Epoch 5/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.0417 - categorical_accuracy: 0.9881 - val_loss: 2.0857 - val_categorical_accuracy: 0.6921
Epoch 6/20
10649/10649 [==============================] - 18s 2ms/step - loss: 0.0367 - categori

In [24]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 19s 2ms/step - loss: 1.2478 - categorical_accuracy: 0.6103 - val_loss: 1.1523 - val_categorical_accuracy: 0.5952
Epoch 2/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.6521 - categorical_accuracy: 0.7896 - val_loss: 1.0691 - val_categorical_accuracy: 0.6940
Epoch 3/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.3369 - categorical_accuracy: 0.8916 - val_loss: 1.5083 - val_categorical_accuracy: 0.6583
Epoch 4/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.1657 - categorical_accuracy: 0.9469 - val_loss: 1.7315 - val_categorical_accuracy: 0.6857
Epoch 5/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.0981 - categorical_accuracy: 0.9701 - val_loss: 1.8432 - val_categorical_accuracy: 0.6928
Epoch 6/20
10649/10649 [==============================] - 17s 2ms/step - loss: 0.0761 - categori